# Surveying Telescopes

Analysing data from an optical interferometer to determine the 3-dimensional locations of the telescopes.

Data are a complete year (2012) of optical path delay observations from the CHARA interferometer in California (see http://www.chara.gsu.edu).

In [2]:
import sys
import pandas as pd
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

In [26]:
# function to load file
def load_file(filename):
    try:
        f = open(filename,'rb')
    except IOError:
        print('Cannot open',filename)
        sys.exit()
    with f:
        return pd.read_csv(f)